In [1]:
import tensorflow as tf
import pandas as pd
import json
import configparser
from pathlib import Path
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')
data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses previstos

# Prever Cesta

In [ ]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano', 'salario', 'razao'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[::, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(LSTM(32, stateful=True, input_shape=(look_back, 1), return_sequences=True, batch_input_shape=(1, look_back, 1)))
model.add(LSTM(32,stateful=True))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_LSTM_{objeto.lower()}_{regiao.lower()}.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes_cesta' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

# Prever Produtos

In [2]:
path_inicial = r'C:\\Users\\mathe\\Documents\\NotebooksMateriais\\previsao_cestas\\datasets_produtos\\'

regioes = ['ilheus','itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite', 'manteiga', 'oleo', 'pao', 'tomate']

for regiao in regioes:
    for objeto in produtos:
        #Carrega os dados
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        cesta = cesta.iloc[::]

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(LSTM(32, stateful=True, input_shape=(look_back, 1), return_sequences=True, batch_input_shape=(1, look_back, 1)))
        model.add(LSTM(32,stateful=True))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_val, batch_size=1)

        resultados = [valor for valor in forecast[0]]
        objeto = objeto.replace(" ", '_')
        output = {objeto.lower():str(resultados)}
        filename = f"previsao_LSTM_{objeto.lower()}_{regiao.lower()}.json"
        pasta_atual = os.getcwd()
        full_url = Path(pasta_atual) / 'resultados' / 'previsoes_produtos' / f'{regiao}' /filename
        os.makedirs(os.path.dirname(full_url), exist_ok=True)
        with open(full_url,"w") as file:
            json.dump(output,file,ensure_ascii=False)
        print("Arquivo com resultados gerados")

Epoch 1/150
197/197 [==============================] - 1s 3ms/step - loss: 1.5056e-06
Epoch 2/150
197/197 [==============================] - 1s 3ms/step - loss: 2.4743e-06
Epoch 3/150
197/197 [==============================] - 1s 3ms/step - loss: 3.0748e-06
Epoch 4/150
197/197 [==============================] - 1s 3ms/step - loss: 3.0484e-06
Epoch 5/150
197/197 [==============================] - 1s 3ms/step - loss: 2.9421e-06
Epoch 6/150
197/197 [==============================] - 1s 3ms/step - loss: 2.8102e-06
Epoch 7/150
197/197 [==============================] - 1s 3ms/step - loss: 3.0436e-06
Epoch 8/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3487e-06
Epoch 9/150
197/197 [==============================] - 1s 3ms/step - loss: 3.4488e-06
Epoch 10/150
197/197 [==============================] - 1s 3ms/step - loss: 3.5792e-06
Epoch 11/150
197/197 [==============================] - 1s 3ms/step - loss: 3.5712e-06
Epoch 12/150
197/197 [==============================

197/197 [==============================] - 1s 3ms/step - loss: 5.2787e-06
Epoch 95/150
197/197 [==============================] - 1s 3ms/step - loss: 5.2950e-06
Epoch 96/150
197/197 [==============================] - 1s 3ms/step - loss: 5.2991e-06
Epoch 97/150
197/197 [==============================] - 1s 3ms/step - loss: 5.2820e-06
Epoch 98/150
197/197 [==============================] - 1s 3ms/step - loss: 5.2395e-06
Epoch 99/150
197/197 [==============================] - 1s 3ms/step - loss: 5.2610e-06
Epoch 100/150
197/197 [==============================] - 1s 3ms/step - loss: 5.3624e-06
Epoch 101/150
197/197 [==============================] - 1s 3ms/step - loss: 5.4180e-06
Epoch 102/150
197/197 [==============================] - 1s 3ms/step - loss: 5.4151e-06
Epoch 103/150
197/197 [==============================] - 1s 3ms/step - loss: 5.3862e-06
Epoch 104/150
197/197 [==============================] - 1s 3ms/step - loss: 5.2065e-06
Epoch 105/150
197/197 [============================

197/197 [==============================] - 1s 3ms/step - loss: 8.0443e-06
Epoch 37/150
197/197 [==============================] - 1s 3ms/step - loss: 7.8144e-06
Epoch 38/150
197/197 [==============================] - 1s 3ms/step - loss: 8.0432e-06
Epoch 39/150
197/197 [==============================] - 1s 3ms/step - loss: 8.0113e-06
Epoch 40/150
197/197 [==============================] - 1s 3ms/step - loss: 8.0407e-06
Epoch 41/150
197/197 [==============================] - 1s 3ms/step - loss: 7.7313e-06
Epoch 42/150
197/197 [==============================] - 1s 3ms/step - loss: 7.9763e-06
Epoch 43/150
197/197 [==============================] - 1s 3ms/step - loss: 7.9708e-06
Epoch 44/150
197/197 [==============================] - 1s 3ms/step - loss: 8.2008e-06
Epoch 45/150
197/197 [==============================] - 1s 3ms/step - loss: 8.4364e-06
Epoch 46/150
197/197 [==============================] - 1s 3ms/step - loss: 8.4502e-06
Epoch 47/150
197/197 [==============================] - 

197/197 [==============================] - 1s 3ms/step - loss: 9.9435e-06
Epoch 130/150
197/197 [==============================] - 1s 3ms/step - loss: 9.9589e-06
Epoch 131/150
197/197 [==============================] - 1s 3ms/step - loss: 9.9358e-06
Epoch 132/150
197/197 [==============================] - 1s 3ms/step - loss: 9.9621e-06
Epoch 133/150
197/197 [==============================] - 1s 3ms/step - loss: 9.9594e-06
Epoch 134/150
197/197 [==============================] - 1s 3ms/step - loss: 9.9812e-06
Epoch 135/150
197/197 [==============================] - 1s 3ms/step - loss: 1.0043e-05
Epoch 136/150
197/197 [==============================] - 1s 3ms/step - loss: 1.0063e-05
Epoch 137/150
197/197 [==============================] - 1s 3ms/step - loss: 1.0029e-05
Epoch 138/150
197/197 [==============================] - 1s 3ms/step - loss: 9.9528e-06
Epoch 139/150
197/197 [==============================] - 1s 3ms/step - loss: 1.0004e-05
Epoch 140/150
197/197 [=======================

197/197 [==============================] - 1s 3ms/step - loss: 4.0677e-05
Epoch 73/150
197/197 [==============================] - 1s 3ms/step - loss: 4.0484e-05
Epoch 74/150
197/197 [==============================] - 1s 3ms/step - loss: 4.0337e-05
Epoch 75/150
197/197 [==============================] - 1s 3ms/step - loss: 4.0151e-05
Epoch 76/150
197/197 [==============================] - 1s 3ms/step - loss: 4.0019e-05
Epoch 77/150
197/197 [==============================] - 1s 3ms/step - loss: 3.9842e-05
Epoch 78/150
197/197 [==============================] - 1s 3ms/step - loss: 3.9724e-05
Epoch 79/150
197/197 [==============================] - 1s 3ms/step - loss: 3.9554e-05
Epoch 80/150
197/197 [==============================] - 1s 3ms/step - loss: 3.9449e-05
Epoch 81/150
197/197 [==============================] - 1s 3ms/step - loss: 3.9286e-05
Epoch 82/150
197/197 [==============================] - 1s 3ms/step - loss: 3.9192e-05
Epoch 83/150
197/197 [==============================] - 

...

197/197 [==============================] - 1s 3ms/step - loss: 3.3045e-06
Epoch 29/150
197/197 [==============================] - 1s 3ms/step - loss: 3.2511e-06
Epoch 30/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3262e-06
Epoch 31/150
197/197 [==============================] - 1s 3ms/step - loss: 3.2812e-06
Epoch 32/150
197/197 [==============================] - 1s 3ms/step - loss: 3.2951e-06
Epoch 33/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3123e-06
Epoch 34/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3311e-06
Epoch 35/150
197/197 [==============================] - 1s 3ms/step - loss: 3.2930e-06A: 0s - loss: 6.1
Epoch 36/150
197/197 [==============================] - 1s 3ms/step - loss: 3.1648e-06
Epoch 37/150
197/197 [==============================] - 1s 3ms/step - loss: 3.1437e-06
Epoch 38/150
197/197 [==============================] - 1s 3ms/step - loss: 3.2069e-06A: 0s - loss: 5.10
Epoch 39/150
197/197 

197/197 [==============================] - 1s 3ms/step - loss: 3.3335e-06
Epoch 122/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3419e-06
Epoch 123/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3951e-06
Epoch 124/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3885e-06
Epoch 125/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3358e-06
Epoch 126/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3047e-06
Epoch 127/150
197/197 [==============================] - 1s 3ms/step - loss: 3.3692e-06
Epoch 128/150
197/197 [==============================] - 1s 3ms/step - loss: 3.4241e-06
Epoch 129/150
197/197 [==============================] - 1s 3ms/step - loss: 3.4273e-06
Epoch 130/150
197/197 [==============================] - 1s 3ms/step - loss: 3.4411e-06
Epoch 131/150
197/197 [==============================] - 1s 4ms/step - loss: 3.4240e-06
Epoch 132/150
197/197 [=======================

197/197 [==============================] - 1s 3ms/step - loss: 1.3061e-05
Epoch 55/150
197/197 [==============================] - 1s 3ms/step - loss: 1.2892e-05
Epoch 56/150
197/197 [==============================] - 1s 3ms/step - loss: 1.2728e-05
Epoch 57/150
197/197 [==============================] - 1s 3ms/step - loss: 1.2554e-05A: 0s - loss: 1.627
Epoch 58/150
197/197 [==============================] - 1s 3ms/step - loss: 1.2423e-05
Epoch 59/150
197/197 [==============================] - 1s 3ms/step - loss: 1.2261e-05
Epoch 60/150
197/197 [==============================] - 1s 3ms/step - loss: 1.2090e-05A: 0s - loss:
Epoch 61/150
197/197 [==============================] - 1s 3ms/step - loss: 1.1997e-05
Epoch 62/150
197/197 [==============================] - 1s 3ms/step - loss: 1.1932e-05
Epoch 63/150
197/197 [==============================] - 1s 3ms/step - loss: 1.1875e-05
Epoch 64/150
197/197 [==============================] - 1s 3ms/step - loss: 1.1839e-05A: 0s - loss: 1.
Epoch 65

197/197 [==============================] - 1s 3ms/step - loss: 9.6750e-06
Epoch 147/150
197/197 [==============================] - 1s 3ms/step - loss: 9.6968e-06
Epoch 148/150
197/197 [==============================] - 1s 3ms/step - loss: 9.7066e-06
Epoch 149/150
197/197 [==============================] - 1s 3ms/step - loss: 9.7077e-06
Epoch 150/150
197/197 [==============================] - 1s 3ms/step - loss: 9.7030e-06
Arquivo com resultados gerados
Epoch 1/150
197/197 [==============================] - 1s 3ms/step - loss: 1.8771e-04
Epoch 2/150
197/197 [==============================] - 1s 3ms/step - loss: 2.4903e-04
Epoch 3/150
197/197 [==============================] - 1s 3ms/step - loss: 2.3217e-04
Epoch 4/150
197/197 [==============================] - 1s 3ms/step - loss: 2.3718e-04
Epoch 5/150
197/197 [==============================] - 1s 3ms/step - loss: 2.2773e-04
Epoch 6/150
197/197 [==============================] - 1s 3ms/step - loss: 2.3182e-04
Epoch 7/150
197/197 [======

197/197 [==============================] - 1s 4ms/step - loss: 1.2703e-04
Epoch 80/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2673e-04
Epoch 81/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2646e-04
Epoch 82/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2621e-04
Epoch 83/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2597e-04
Epoch 84/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2573e-04
Epoch 85/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2553e-04
Epoch 86/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2530e-04
Epoch 87/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2512e-04
Epoch 88/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2494e-04
Epoch 89/150
197/197 [==============================] - 1s 4ms/step - loss: 1.2477e-04
Epoch 90/150
197/197 [==============================] - 